# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [15]:
# import required libraries
import numpy as np
import pandas as pd
import zipfile

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [16]:
# your code here
  
# read and import
orders = pd.read_csv('Orders.zip',compression='zip')
  
# display dataset
orders.head()


,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [17]:
# your code here

#Sub Problem 1: How to aggregate the amount_spent for unique customers?

total_spent_costumer = orders.groupby(['CustomerID']).sum()
total_spent_costumer

,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,,
12346,61619,541431,2011,1,2,10,74215,1.04,77183.60
12347,42441700,101296926,365971,1383,441,2219,2458,481.21,4310.00
12348,2807120,16869685,62324,257,111,472,2341,178.71,1797.24
12349,35444274,42165457,146803,803,73,657,631,605.10,1757.55
12350,1365627,9231629,34187,34,51,272,197,65.30,334.40
...,...,...,...,...,...,...,...,...,...
18280,1110495,5457120,20110,30,10,90,45,47.65,180.60
18281,1560699,3895248,14077,42,49,70,54,39.36,80.82
18282,4642134,6838540,24132,116,60,146,103,62.39,178.05


In [19]:
# Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?
# Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

vip_quantile = total_spent_costumer.quantile([.95])  #give us the 95quantile
pref_quantile = total_spent_costumer.quantile([.75])  #give us the 75quantile

#Create buckets of customers 

total_spent_costumer['costumer_type'] = pd.qcut(total_spent_costumer['amount_spent'], q=[0,0.75,0.95,1] , labels = ['Normal_customers','Preferred_customers', 'VIP_customers'])

total_spent_costumer


,Unnamed: 0,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent,costumer_type
CustomerID,,,,,,,,,,
12346,61619,541431,2011,1,2,10,74215,1.04,77183.60,VIP_customers
12347,42441700,101296926,365971,1383,441,2219,2458,481.21,4310.00,Preferred_customers
12348,2807120,16869685,62324,257,111,472,2341,178.71,1797.24,Preferred_customers
12349,35444274,42165457,146803,803,73,657,631,605.10,1757.55,Preferred_customers
12350,1365627,9231629,34187,34,51,272,197,65.30,334.40,Normal_customers
...,...,...,...,...,...,...,...,...,...,...
18280,1110495,5457120,20110,30,10,90,45,47.65,180.60,Normal_customers
18281,1560699,3895248,14077,42,49,70,54,39.36,80.82,Normal_customers
18282,4642134,6838540,24132,116,60,146,103,62.39,178.05,Normal_customers


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [32]:
# your code here

# Merge the information from both df
customer_country = orders.merge(total_spent_costumer,left_on = 'CustomerID',right_on = 'CustomerID')
customer_country

# Select the VIP customers
vip_join = customer_country[(customer_country['costumer_type'] == 'VIP_customers')]
vip_join

# Sort VIP customers by country and count the total of customer in each
country_vip = vip_join.groupby(['Country']).agg({'CustomerID':'count'})
country_vip

# See the countries with more clients (top 5)
countries = country_vip.sort_values(["CustomerID"], ascending=[False])
countries.head(5)   #UK (we can also use head(1) and see only the first place)

,CustomerID
Country,
United Kingdom,84185
EIRE,7077
France,3290
Germany,3127
Netherlands,2080


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [29]:
# your code here

# Select only Vip and Preferred customers
vip_pref_join = customer_country[(customer_country['costumer_type'] == 'VIP_customers') | (customer_country['costumer_type'] == 'Preferred_customers')]
vip_pref_join

# Sort by country and count the total of customers
country_vip_pref = vip_pref_join.groupby(['Country']).agg({'CustomerID':'count'})
country_vip_pref

# See the countries with more clients (top 5)
countries_2 = country_vip_pref.sort_values(["CustomerID"], ascending=[False])
countries_2.head(5)   #UK


,CustomerID
Country,
United Kingdom,221635
Germany,7349
EIRE,7238
France,6301
Netherlands,2080
